In [3]:
from pathlib import Path
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

RES_FILE = Path('results_basic.csv')

In [4]:
res_df = pd.read_csv(RES_FILE)

In [48]:
res_df

,name,table,run_name,time
0,gb_filt_sel_0,tpch_lineitem_1g_star_idx_10,run_0,0.027474
1,gb_filt_sel_0,tpch_lineitem_1g_star_idx_10,run_1,0.041474
2,gb_filt_sel_0,tpch_lineitem_1g_star_idx_10,run_2,0.020661
3,gb_filt_sel_0,tpch_lineitem_1g_star_idx_10,run_3,0.020788
4,gb_filt_sel_0,tpch_lineitem_1g_star_idx_10,run_4,0.033759
...,...,...,...,...
835,distinct,tpch_lineitem_1g_star_idx_10000,run_5,0.023776
836,distinct,tpch_lineitem_1g_star_idx_10000,run_6,0.022142
837,distinct,tpch_lineitem_1g_star_idx_10000,run_7,0.019235
838,distinct,tpch_lineitem_1g_star_idx_10000,run_8,0.018291


In [5]:
def handle_sel_results(res_df: pd.DataFrame):
    # filter res_df by table name
    # split df into multiple dfs by table name
    sel_df = res_df[res_df['name'].str.contains('sel')].copy()
    sel_df['sel'] = (sel_df['name'].str.rsplit('_', n=1).str[-1]).astype(int)
    lg = go.Figure()
    tablenames = sel_df['table'].unique()
    for table in tablenames:
        df = sel_df[sel_df['table'] == table]
        df = df.sort_values(by=['sel', 'run_name'])
        times: pd.DataFrame = df.groupby('sel', as_index=False)['time'].min()
        # only keep multiples of 20
        # times = times.loc[times['sel'] % 20 == 0]
        print(df, times)
        lg.add_trace(go.Line(x=times['sel'], y=times['time'], name=table.removeprefix('tpch_lineitem_')))
        # break
        # # fig = px.line(df, x='size', y='time', title=f'{table} - Time vs Size')
        # # fig.show()
    lg.show()


In [ ]:
def st_basic_res(res_df: pd.DataFrame):
    res_df = res_df[~res_df['name'].str.contains('sel')].copy()
    res_df = res_df[res_df['name'].isin(['agg', 'distinct', 'filt_1', 'filt_3', 'gb_1'])]
    tablenames = res_df['table'].unique()
    fig = go.Figure()
    for table in tablenames:
        if "1g" in table:
            # skip 1g for now
            continue
        df = res_df[res_df['table'] == table]
        df = df.sort_values(by=['name', 'run_name'])
        times: pd.DataFrame = df.groupby('name', as_index=False)['time'].mean()
        print(df, times)
        fig.add_trace(go.Line(x=times['name'], y=times['time'], name=table.removeprefix('tpch_lineitem_')))
    fig.show()

In [59]:
def basic_res(res_df: pd.DataFrame):
    res_df = res_df[~res_df['name'].str.contains('sel')].copy()
    res_df = res_df[res_df['name'].isin(['agg', 'distinct', 'filt_1', 'filt_3', 'gb_1'])]
    # replace gb_1 with gb and filt_3 with filt_2
    res_df['name'] = res_df['name'].replace({'gb_1': 'gb', 'filt_3': 'filt_2'})
    st_table = "tpch_lineitem_10g"
    no_idx_table = "tpch_lineitem_10g_no_idx"

    no_idx_df = res_df[res_df['table'] == no_idx_table]
    no_idx_df = no_idx_df.sort_values(by=['name', 'run_name'])
    no_idx_times: pd.DataFrame = no_idx_df.groupby('name', as_index=False)['time'].mean()
    
    st_df = res_df[res_df['table'] == st_table]
    st_df = st_df.sort_values(by=['name', 'run_name'])
    st_times: pd.DataFrame = st_df.groupby('name', as_index=False)['time'].mean()
    speedup = no_idx_times['time'] / st_times['time']
    print(st_times, no_idx_times, speedup)
    fig = px.bar(x=st_times['name'], y=speedup, labels={'x': 'Query', 'y': 'Speedup'}, title='Speedup of StarTree vs No Index')
    fig.show()
    fig.update_layout(title="", margin=go.layout.Margin(
            l=5, #left margin
            r=5, #right margin
            b=5, #bottom margin
            t=5  #top margin
        )
    )
    img = fig.to_image(format="pdf", width=400, height=300)
    with open('st_speedup.pdf', 'wb') as f:
        f.write(img)



In [60]:
basic_res(res_df)

       name      time
0       agg  0.024960
1  distinct  0.030302
2    filt_1  1.024111
3    filt_2  0.023830
4        gb  0.022517        name      time
0       agg  1.010281
1  distinct  0.022088
2    filt_1  1.104487
3    filt_2  0.382184
4        gb  0.577332 0    40.475507
1     0.728929
2     1.078483
3    16.038199
4    25.639836
Name: time, dtype: float64


          name                         table run_name      time
460        agg  tpch_lineitem_1g_star_idx_10    run_0  0.034640
461        agg  tpch_lineitem_1g_star_idx_10    run_1  0.028806
462        agg  tpch_lineitem_1g_star_idx_10    run_2  0.027788
463        agg  tpch_lineitem_1g_star_idx_10    run_3  0.020898
464        agg  tpch_lineitem_1g_star_idx_10    run_4  0.030850
..         ...                           ...      ...       ...
525  gb_filt_2  tpch_lineitem_1g_star_idx_10    run_5  0.024271
526  gb_filt_2  tpch_lineitem_1g_star_idx_10    run_6  0.034746
527  gb_filt_2  tpch_lineitem_1g_star_idx_10    run_7  0.023344
528  gb_filt_2  tpch_lineitem_1g_star_idx_10    run_8  0.028786
529  gb_filt_2  tpch_lineitem_1g_star_idx_10    run_9  0.049952

[100 rows x 4 columns]         name      time
0        agg  0.020898
1   distinct  0.018808
2     filt_1  0.117274
3     filt_2  0.049854
4     filt_3  0.019247
5     filt_4  0.019280
6       gb_1  0.022757
7       gb_2  0.021300
8

# StarTree Index sizes

In [62]:
def extract_st_idx_size(file:Path):
    """Takes CSV file of the form:
    Property,Sizes
    0.null.STAR_TREE.SIZE,"1438573, 309865, 2961549"
    0.l_receiptdate.FORWARD_INDEX.SIZE,"193607, 94334, 315228
    ...
    First column is the property name (table column)
    And second column is the size of each segment on that server

    Returns sum of all sizes across all segments
    """
    df = pd.read_csv(file)
    sizes = df['Sizes']
    sizes = sizes.str.split(', ')
    sizes = sizes.apply(lambda x: sum(map(int, x)))
    return sizes.sum()
    

In [63]:
def extract_threshold(name:str):
    """1g_star_idx_10_sizes_server1.csv -> 10"""
    return int(name.split('_')[3])

In [64]:
from collections import defaultdict

In [65]:
STAR_TREE_SIZES_DIR = Path('results/star_tree_index_size_T/')
assert STAR_TREE_SIZES_DIR.exists()

sizes = defaultdict(int)

for server_dir in STAR_TREE_SIZES_DIR.iterdir():
    server_name = server_dir.name
    print(server_name)
    for file in server_dir.iterdir():
        if "index_map" in file.name:
            continue
        threshold = extract_threshold(file.name)
        size = extract_st_idx_size(file)
        sizes[threshold] += size

sizes = dict(sizes)
        

csv_out_star_T_server_2
csv_out_star_T_server_1


In [66]:
fig = px.bar(x=list(sizes.keys()), y=list(sizes.values()))
fig.update_xaxes(type='category', title='Threshold', rangemode='tozero')
fig.update_yaxes(title='Index Size (bytes)', rangemode='tozero')
fig.update_layout(title="", margin=go.layout.Margin(
        l=5, #left margin
        r=5, #right margin
        b=5, #bottom margin
        t=5  #top margin
    )
)
img = fig.to_image(format="pdf", width=400, height=300)
with open('star_tree_index_size.pdf', 'wb') as f:
    f.write(img)